# Value Function Iteration

This notebook takes us through the household maximization problem.

In [ ]:
# Load necessary packages and scripts
import matplotlib.pyplot as plt
import numba as nb
import numpy as np
import tqdm

from scipy.interpolate import interp1d
from scipy.stats import linregress

The code block below provides the logic for the value function iteration procedure. While it can prove helpful and instructive to go through it, it is not necessary for you to understand this code.

The arguments of the function VFI are model parameters such as the interest rate. The output of the function are the policy functions for consumption `c[a, y, t]` and assets `a_t+1[a, y, t]` as well as the asset grid `a_grid`.

In [ ]:
# @title Value Function Iteration Code

# Backwards induction using Value Function Iteration
# @nb.jit(nopython=True)
def VFI(T, util_type, eta, beta, r, gamma, sigma_y, cbar=100, bonus_question=False):

    # Gross interest rate
    R = 1 + r

    # Transition matrix for income process
    P = np.array([[1+gamma, 1-gamma], [1-gamma, 1+gamma]])/2

    # Income shock states
    Y = np.array([[1-sigma_y, 1+sigma_y]])

    # Set up a grid for the asset state variable
    if bonus_question:
        a_min = -6*(1-sigma_y)/R
        a_max = 15
        step_size = 0.005
        cbar=50
    else:
        a_min = -1.5*(1-sigma_y)/R
        a_max = 4
        step_size = 0.001

    a_grid = np.arange(a_min, a_max, step_size).reshape((-1, 1))

    # Initialize policy functions
    c_pol = np.zeros((len(a_grid), 2, T))
    a_prime_pol = np.zeros((len(a_grid), 2, T))

    # Last period decision, i.e., V(a, y, T)
    v = []
    c_last = R * a_grid + Y
    c_neg = c_last <= 0
    v=util(c_last, util_type, eta, cbar)
    v[c_neg] = -1e12

    for i in tqdm.tqdm(range(T)):
        tempv = np.zeros((len(a_grid), 2))

        for (j, a) in enumerate(a_grid):
            # Implied value of the value function
            b = np.linspace(a_min, a_max - step_size, len(a_grid)).reshape((-1, 1))
            K = util(R * a - b + Y, util_type, eta, cbar) + beta * (P @ v.T).T

            # Find index up to which we compute the max
            ind = int(max(1, min(
                round_away((min(a_max, R * a + Y[0, 0]) - a_min) / step_size),
                K.shape[0] - 1
            )))

            # Obtain the maximum value of all potential choices (avoid axis arg for numba)
            M = [np.max(K[:ind, 0]), np.max(K[:ind, 1])]
            I = [np.argmax(K[:ind, 0]), np.argmax(K[:ind, 1])]

            # Extract value and save decision
            tempv[j, :] = M
            a_prime_pol[j, :, T-i-1] = b[I].flatten() # a_grid[I].flatten()
            c_pol[j, :, T-i-1] = R * a_grid[j] + Y - a_prime_pol[j, :, T-i-1]

        # Update value function
        v = tempv.copy()

    return c_pol, a_prime_pol, a_grid.flatten()

def crra_util(c, eta):
    return (c**(1-eta)-1)/(1-eta)

def quadratic_util(c, cbar):
    return -(c - cbar)**2

def util(c, util_type, eta, cbar):
    if util_type.lower() == 'crra':
        return crra_util(c, eta)
    elif util_type.lower() == 'quadratic':
        return quadratic_util(c, cbar)
    else:
        raise ValueError("Unsupported utility type")

# Rounding to integer, away from zero in Python
def round_away(x):
    a = np.abs(x)
    r = np.floor(a) + np.floor(2 * (a % 1))
    return np.sign(x) * r

## Default Parametrization

In [ ]:
# Default Parametrization
# -----------------------

T = 45              # Number of periods (Length of the individual's life cycle)

util_type = 'CRRA'  # Utility Function type (CRRA or Quadratic)
eta = 2             # Coefficient of Relative Risk Aversion

beta = 0.90     # Discount Factor
r = 0.04            # Interest Rate  (Real, Risk-free)
R = 1+r           # Gross Interest Rate

gamma = 0.40  # Income Shock Persistence
sigma_y = 0.20       # Income Shock Volatility

### Question 4: Compare CRRA and Quadratic Utility

1. Using the default parametrization, obtain the policy function for consumption by calling the function VFI for both specifications of the utility function.
2. Plot both consumption policy functions in the first period for both income states. On the x-axis should be the grid for `a`, on the y-axis `c[a, y1, 1]` and `c[a, y2, 1]` for the respective preference specification.
3. Explain the economic intuition behind the differences of the consumption policy functions.

In [ ]:
# Set the utility type to CRRA
u_type='CRRA'

# Run VFI with CRRA utility and store the policy functions
c_pol_1, a_prime_pol_1, a_grid = VFI(T=T,
                                     util_type=u_type, eta=eta,
                                     beta=beta, r=r,
                                     gamma=gamma, sigma_y=sigma_y)

# The progress window below illustrates the evolving numerical solution progress for the life cycle problem.

In [ ]:
# Set the utility type to Quadratic
# u_type =

# Run VFI with quadratic utility and store the policy functions
c_pol_2, a_prime_pol_2, a_grid = VFI(T=T,
                                     util_type=u_type, eta=eta,
                                     beta=beta, r=r,
                                     gamma=gamma, sigma_y=sigma_y)

In [ ]:
# Set period to the first model period. Remember that Python starts indexing at 0, not at 1.
# period =

# Plot the consumption policy functions
fig, axs = plt.subplots(1, 2, figsize=(12, 8))

# Plots for first income state
axs[0].plot(a_grid, c_pol_1[:, 0, period], label='CRRA', color='black')
axs[0].plot(a_grid, c_pol_2[:, 0, period], label='Quadratic', color='red')
axs[0].set_title('First Income State')

# Plots for second income state
axs[1].plot(a_grid, c_pol_1[:, 1, period], label='CRRA', color='black')
axs[1].plot(a_grid, c_pol_2[:, 1, period], label='Quadratic', color='red')
axs[1].set_title('Second Income State')

# Common plot aesthetics
for ax in axs:
    ax.grid(True)
    ax.legend()
    ax.set_ylabel('Consumption')
    ax.set_xlabel('Assets')

### Question 5: Simulated Life Cycle Profiles

1. Generate a time profile of consumption by choosing initial assets equal to 0 and by using the policy functions `c[a, y, t]` and `a_t+1[a, y, t]`.
2. Simulate the income process starting at `y_1 = 1+sigma_y` and initializing the random number generator using `np.random.seed(23)`.
3. Compare the variance of the income and the consumption path. How do they compare and what is the economic interpretation?

In [ ]:
# Income shock states
Y = [1-sigma_y, 1+sigma_y]
# Transition matrix for income process
P = np.array([[1+gamma, 1-gamma], [1-gamma, 1+gamma]])/2

# Random component
np.random.seed(23)
r_num = np.random.rand(T)

In [ ]:
# Set initial income and assets
# y_initial =
# a_initial =


# Initialize lists for the income process
y_state = [y_initial]
y_t = [Y[y_state[0]]]

# Simulate income process
for k in range(T):
    y_state.append(int(r_num[k] > P[y_state[k], 0])) # A Markov income process for y[t]
    y_t.append(Y[y_state[k + 1]])                    # Income at time t

# Set up list for consumption c_t
c_t = []
# Set initial assets to zero
a_t = [a_initial]

# Simulate paths for assets a_t and consumption c_t
for it in range(T):
    a_t.append(float(interp1d(a_grid.flatten(), a_prime_pol_1[:, y_state[it], it])(a_t[it])))
    c_t.append(R*a_t[it] + y_t[it] - a_t[it + 1])

In [ ]:
# Plot the time series for consumption , income, and assets
fig, axs = plt.subplots(1, 2, figsize=(12, 8))

# Plot consumption and assets over time
axs[0].plot(c_t, label='Consumption', color='black')
axs[1].plot(a_t, label='Assets', color='black')

# Add a span to display high income (bit of a tricky code...)
i, j = 0, 0
add_label = True
while i < len(y_state):
    j = i + 1
    if y_state[i] == 1:
        while j < len(y_state) and y_state[j] == 1:
            j += 1
        if add_label:
            axs[0].axvspan(i, j, alpha=0.2, label='High Income State')
            axs[1].axvspan(i, j, alpha=0.2, label='High Income State')
            add_label = False
        else:
            axs[0].axvspan(i, j, alpha=0.2)
            axs[1].axvspan(i, j, alpha=0.2)
    i = j


for ax in axs:
    ax.set_xlabel('Period')
    ax.legend()

# Calculate the variance of consumption as a share of the variance of income
np.var(c_t) / np.var(y_t)

### Question 6: Comparative Statics
1. How does the consumption policy function `c[a,y1,1]` change if you assume CRRA utility change and beta increases from `beta=0.90` to `beta=0.95`?
2. How does it change if gamma increases from `gamma=0.1` to `gamma=0.9`?
3. Explain the respective economic intuition.

In [ ]:
# Change in patience

# Insert the paramter values for comparison
# beta1 =
# beta2 =


# Run VFI with low beta
c_beta_l, a_beta_l, a_grid = VFI(T=T,
                                 util_type='CRRA', eta=eta,
                                 beta=beta1, r=r,
                                 gamma=gamma, sigma_y=sigma_y)

# Run VFI with high beta
c_beta_h, a_beta_h, a_grid = VFI(T=T,
                                 util_type='CRRA', eta=eta,
                                 beta=beta2, r=r,
                                 gamma=gamma, sigma_y=sigma_y)

In [ ]:
# Change in income shock variance

# Insert the paramter values for comparison
# gamma1 =
# gamma2 =


# Run VFI with low gamma
c_gamma_l, a_gamma_l, a_grid = VFI(T=T,
                                 util_type='CRRA', eta=eta,
                                 beta=beta, r=r,
                                 gamma=gamma1, sigma_y=sigma_y)

# Run VFI with high gamma
c_gamma_h, a_gamma_h, a_grid = VFI(T=T,
                                 util_type='CRRA', eta=eta,
                                 beta=beta, r=r,
                                 gamma=gamma2, sigma_y=sigma_y)

In [ ]:
# Plot c_pol for comparative statics
fig, axs = plt.subplots(1, 2, figsize=(12, 8))

axs[0].plot(a_grid, c_beta_l[:, 0, 0], label=f'Low Beta (β={beta1})', color='black')
axs[0].plot(a_grid, c_beta_h[:, 0, 0], label=f'High Beta (β={beta2})', color='red')
axs[0].set_title('Change in Patience')

axs[1].plot(a_grid, c_gamma_l[:, 0, 0], label=f'Low Gamma (γ={gamma1})', color='black')
axs[1].plot(a_grid, c_gamma_h[:, 0, 0], label=f'High Gamma (γ={gamma2})', color='red')
axs[1].set_title('Change in Income Shock Variance')

for ax in axs:
    ax.set_xlabel('')
    ax.set_ylabel('Consumption')
    ax.legend()
    ax.grid()

### Question 7: Bonus Question

Regress consumption growth on current income to evaluate Hall's random walk hypothesis.
Simulate the life cycle profiles of N=10,000 individuals using the specification with a quadratic utility function. Calculate consumption growth rates from the simulated time series. Set `beta=1/(1+r)` and T=60. Discard the first and last p=10 periods before you perform the regression. Provide an economic interpretation of the regression results (coefficients, p-values, `R^2`).

In [ ]:
# Set the number of periods to simulate
# T_rw =

# Assign parameter values
gamma = 0.7          # Income Shock Persistence
sigma_y = 0.20       # Income Shock Volatility
r = 0.04
R = 1+r

# Set beta and utility type
# beta_bonus =
# u_type =

# Run VFI for bonus question
c_pol, a_prime_pol, a_grid = VFI(T=T_rw,
                                 util_type=u_type, eta=eta,
                                 beta=beta_bonus, r=r,
                                 gamma=gamma, sigma_y=sigma_y,
                                 bonus_question=True)

In [ ]:
# Set the number of individuals to simulate
# N =
# Set number of periods to discard (before AND after, i.e., we discard 2p periods)
# p =

y_all = np.zeros((N, T_rw))
c_all = np.zeros((N, T_rw))
a_all = np.zeros((N, T_rw))
c_all_growth = np.zeros((N, T_rw - 2*p))
y_all_reg = np.zeros((N, T_rw - 2*p))

for iN in tqdm.tqdm(range(N)):
    y_state = [1]
    y_t = [Y[y_state[0]]]

    np.random.seed(23+iN)
    r_num = np.random.rand(T_rw)

    for k in range(T_rw - 1):
        y_state.append(int(r_num[k] > P[y_state[k], 0])) # A Markov income process for y[t]
        y_t.append(Y[y_state[k + 1]])                    # Income at time t

    # Simulate consumption path
    c_t = []
    a_t = [0]

    for it in range(T_rw):
        a_t.append(float(interp1d(a_grid.flatten(), a_prime_pol[:, y_state[it], it])(a_t[it])))
        c_t.append(R*a_t[it] + y_t[it] - a_t[it + 1])

    # Calculate growth rates
    c_growth = np.exp(np.diff(np.log(c_t))) - 1
    c_growth = c_growth[p:-p+1]
    c_growth[np.abs(c_growth) < 0.0001] = 0

    # Store results
    y_all[iN, :] = y_t
    c_all[iN, :] = c_t
    a_all[iN, :] = a_t[:-1]
    c_all_growth[iN, :] = c_growth
    y_all_reg[iN, :] = y_t[p-2:-p-2]

# Run regression
lm = linregress(y_all_reg.flatten(), c_all_growth.flatten())

# Print results
print(f"\nIntercept           : {lm.intercept:.4f}")
print(f"Slope (coefficient) : {lm.slope:.4f}")
print(f"Standard Error      : {lm.stderr:.4f}")
print(f"p-value             : {lm.pvalue:.4f}")
print(f"R-squared           : {lm.rvalue**2:.4f}")

In [ ]:
# Plot the time series of all individuals
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Plot the data in the first subplot
for i in range(a_all.shape[0]):
    axes[0].plot(a_all[i, :])

# Set the title and labels for the first subplot
axes[0].set_title('Assets')
axes[0].set_xlabel('Period')
axes[0].set_ylabel('')

# Plot the data in the second subplot
for i in range(c_all.shape[0]):
    axes[1].plot(c_all[i, :])

# Set the title and labels for the second subplot
axes[1].set_title('Consumption')
axes[1].set_xlabel('Period')
axes[1].set_ylabel('')

# Plot the data in the third subplot
for i in range(c_all_growth.shape[0]):
    axes[2].plot(c_all_growth[i, :])

# Set the title and labels for the third subplot
axes[2].set_title('Consumption Growth')
axes[2].set_xlabel('Period')
axes[2].set_ylabel('')